In [1]:
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim

import torchdiffeq

from tensorboard_utils import Tensorboard
from tensorboard_utils import tensorboard_event_accumulator

import transformer.Constants as Constants
from transformer.Layers import EncoderLayer, DecoderLayer
from transformer.Modules import ScaledDotProductAttention
from transformer.Models import Decoder, get_attn_key_pad_mask, get_non_pad_mask, get_sinusoid_encoding_table
from transformer.SubLayers import PositionwiseFeedForward

import dataset

import model_process
import checkpoints
from node_transformer import NodeTransformer

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib notebook  
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

print("Torch Version", torch.__version__)

%load_ext autoreload
%autoreload 2

Torch Version 1.1.0


In [2]:
seed = 1
torch.manual_seed(seed)
device = torch.device("cuda")
print("device", device)

device cuda


In [3]:
data = torch.load("/home/mandubian/datasets/multi30k/multi30k.atok.low.pt")

In [4]:
max_token_seq_len = data['settings'].max_token_seq_len
print(max_token_seq_len)

52


In [5]:
train_loader, val_loader = dataset.prepare_dataloaders(data, batch_size=128)

### Create an experiment with a name and a unique ID

In [6]:
exp_name = "node_transformer_full_multi30k"
unique_id = "2019-06-08_1300"

# unique_id = "2019-06-05_1300"
# default
# batch 128
# rtol=1e-3, atol=1e-3
# lr=1e-5

# unique_id = "2019-06-07_2100"
# default
# batch 128
# d_word_vec=64, d_model=64, d_inner=256,
# n_head=4, method='dopri5-ext', rtol=1e-2, atol=1e-2
# lr=1e-4

# unique_id = "2019-06-08_1300"
# default
# batch 128
# rtol=1e-2, atol=1e-2
# lr=1e-4
# progressive depth: 2 (25)
# 0-12: depth 2
# 12-x: depth 6

### Create Model

In [8]:
model = None

In [9]:

src_vocab_sz = train_loader.dataset.src_vocab_size
print("src_vocab_sz", src_vocab_sz)
tgt_vocab_sz = train_loader.dataset.tgt_vocab_size
print("tgt_vocab_sz", tgt_vocab_sz)

if model:
    del model
    
model = NodeTransformer(
    n_src_vocab=max(src_vocab_sz, tgt_vocab_sz),
    n_tgt_vocab=max(src_vocab_sz, tgt_vocab_sz),
    len_max_seq=max_token_seq_len,
    #emb_src_tgt_weight_sharing=False,
    #d_word_vec=64, d_model=64, d_inner=256,
    n_head=8, method='dopri5-ext', rtol=1e-2, atol=1e-2,
    has_node_encoder=True, has_node_decoder=True)

model = model.to(device)

src_vocab_sz 9795
tgt_vocab_sz 17989


### Create Tensorboard metrics logger

In [10]:
tb = Tensorboard(exp_name, unique_name=unique_id)

Writing TensorBoard events locally to runs/node_transformer_full_multi30k_2019-06-08_1300


### Create basic optimizer

In [11]:
optimizer = optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.995), eps=1e-9)


### Train

In [13]:
# Continuous space discretization
timesteps = np.linspace(0., 1, num=6)
timesteps = torch.from_numpy(timesteps).float()

EPOCHS = 50
LOG_INTERVAL = 5

#from torch import autograd
#with autograd.detect_anomaly():
model_process.train(
    exp_name, unique_id,
    model, 
    train_loader, val_loader, timesteps,
    optimizer, device,
    epochs=EPOCHS, tb=tb, log_interval=LOG_INTERVAL,
    start_epoch=19, best_valid_accu=state["acc"]
)

Loaded model and timesteps to cuda


[ Epoch 19 ]
Adding group train to writers (dict_keys([]))
[Training]  loss: 5.9475882889758935, ppl:  382.82895, accuracy: 41.752 %, elapse: 1720832.954ms
Adding group eval to writers (dict_keys(['train']))
[Validation]  loss: 5.292197378259546,  ppl:  198.77974, accuracy: 43.735 %, elapse: 2922.706ms


[ Epoch 20 ]
[Training]  loss: 5.824254621632849, ppl:  338.40880, accuracy: 42.521 %, elapse: 1749549.291ms
[Validation]  loss: 5.302833507392184,  ppl:  200.90527, accuracy: 43.241 %, elapse: 2916.893ms


[ Epoch 21 ]


Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, b

KeyboardInterrupt: 

In [ ]:
# Continuous space discretization
timesteps = np.linspace(0., 1, num=12)
timesteps = torch.from_numpy(timesteps).float()

EPOCHS = 50
LOG_INTERVAL = 5

#from torch import autograd
#with autograd.detect_anomaly():
model_process.train(
    exp_name, unique_id,
    model, 
    train_loader, val_loader, timesteps,
    optimizer, device,
    epochs=EPOCHS, tb=tb, log_interval=LOG_INTERVAL,
    start_epoch=21, best_valid_accu=state["acc"]
)

Loaded model and timesteps to cuda


[ Epoch 21 ]
[Training]  loss: 5.982365558516917, ppl:  396.37691, accuracy: 41.378 %, elapse: 2633471.951ms
[Validation]  loss: 5.288381585692922,  ppl:  198.02268, accuracy: 43.463 %, elapse: 2984.516ms


[ Epoch 22 ]


### Restore best checkpoint (to restart past training)

In [14]:
state = checkpoints.restore_best_checkpoint(
    exp_name, unique_id, "validation", model, optimizer)

print("accuracy", state["acc"])
print("loss", state["loss"])
model = model.to(device)

Extracting state from checkpoints/node_transformer_full_multi30k_2019-06-08_1300_validation_best.pth
Loading model state_dict from state found in checkpoints/node_transformer_full_multi30k_2019-06-08_1300_validation_best.pth
Loading optimizer state_dict from state found in checkpoints/node_transformer_full_multi30k_2019-06-08_1300_validation_best.pth
accuracy 0.4476681710724264
loss 5.363942295853034
